# 加载 wiki 文档查询

## 全局设置

In [1]:
%%time
%%capture

!pip install llama-index-readers-feishu-wiki

CPU times: user 10.1 ms, sys: 3.55 ms, total: 13.7 ms
Wall time: 1.99 s


In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=Ollama(
    base_url="http://ape:11434",
    model="qwen2",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 2.16 s, sys: 228 ms, total: 2.39 s
Wall time: 2 s


## 加载文档

In [43]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [48]:
%%time

app_id = "cli_a644510a88eb900c"
app_secret = "HVk6lCNTQEg22zJkNp7rhehivBUz4GOn"
space_id = "7408151433373335556"

from llama_index.readers.feishu_wiki import FeishuWikiReader

documents=FeishuWikiReader(app_id, app_secret).load_data(space_id=space_id)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): open.feishu.cn:443
Starting new HTTPS connection (1): open.feishu.cn:443
DEBUG:urllib3.connectionpool:https://open.feishu.cn:443 "POST /open-apis/auth/v3/tenant_access_token/internal HTTP/11" 200 102
https://open.feishu.cn:443 "POST /open-apis/auth/v3/tenant_access_token/internal HTTP/11" 200 102
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): open.feishu.cn:443
Starting new HTTPS connection (1): open.feishu.cn:443
DEBUG:urllib3.connectionpool:https://open.feishu.cn:443 "GET /open-apis/wiki/v2/spaces/7408151433373335556/nodes HTTP/11" 200 None
https://open.feishu.cn:443 "GET /open-apis/wiki/v2/spaces/7408151433373335556/nodes HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): open.feishu.cn:443
Starting new HTTPS connection (1): open.feishu.cn:443
DEBUG:urllib3.connectionpool:https://open.feishu.cn:443 "GET /open-apis/docx/v1/documents/FJ1ndY2MsoTWTPxWWaTcXBZonJl/raw_content H

In [49]:
len(documents)

2

In [50]:
documents[0]

Document(id_='9849e2bb-5e16-4e80-880c-cebfa88a6e14', embedding=None, metadata={'document_id': 'FJ1ndY2MsoTWTPxWWaTcXBZonJl'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Docker FAQ\nDocker 加速相关配置\n\ndockerfile 中常用加速\n\n# 使用nvidia作为基础镜像\nFROM nvidia/cuda:12.2.0-devel-ubuntu22.04\n\n# 设置时区环境变量\nENV TZ=Asia/Shanghai\n\n# 设置时区\n# 安装 python3/pip/jupyter相关\nRUN ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && echo $TZ > /etc/timezone \\\n    && sed -i s@/archive.ubuntu.com/@/mirrors.tuna.tsinghua.edu.cn/@g /etc/apt/sources.list \\\n    && apt update && apt install python3 pip git -y \\\n    && apt-get -y install \\\n        pandoc texlive-xetex texlive-fonts-recommended texlive-plain-generic \\\n        latex-cjk-all -y \\\n    && pip config --user set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple \\\n    && pip install \\\n        jupyterlab jupyterlab-language-pack-zh-CN jupyterlab-git jupyterlab-github \\\n        matplotlib \\\n   

## 创建索引

In [51]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents
)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Docker FAQ
Docker 加速相关配置

dockerfile 中常用加速

# 使...
> Adding chunk: Docker FAQ
Docker 加速相关配置

dockerfile 中常用加速

# 使...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: CURRENT_UID=$(id -u):$(id -g) docker-compose up...
> Adding chunk: CURRENT_UID=$(id -u):$(id -g) docker-compose up...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: {
  "repositories": [
    "app-server",
    "cu...
> Adding chunk: {
  "repositories": [
    "app-server",
    "cu...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 首页
在首页你可以对空间进行说明，比如包含以下内容。
🎯  愿景和目标
这里你可以描述知识空间...
> Adding chunk: 首页
在首页你可以对空间进行说明，比如包含以下内容。
🎯  愿景和目标
这里你可以描述知识空间...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ape:11434
Starting new HTTP connection (1): ape:11434
DEBUG:urllib3.connectionpool:http://ape:11434 "POST /api/embeddings HTTP/11" 200 None
http://ape:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connec

## 查询

In [52]:
%%time

query_engine = index.as_query_engine(
    streaming=True
)
streaming_response = query_engine.query("docker容器在后台执行的命令")
streaming_response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ape:11434
Starting new HTTP connection (1): ape:11434
DEBUG:urllib3.connectionpool:http://ape:11434 "POST /api/embeddings HTTP/11" 200 None
http://ape:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 838e3eb1-2149-468b-91c6-f8245f52d571] [Similarity score:             0.642404] Docker FAQ
Docker 加速相关配置

dockerfile 中常用加速

# 使用nvidia作为基础镜像
FROM nvidia/cuda:12.2.0-devel-ubuntu...
> [Node 116c7d5f-11f0-41e4-a6ae-1790250834ee] [Similarity score:             0.574852] CURRENT_UID=$(id -u):$(id -g) docker-compose up -d

docker 容器在后台执行

# tty=true -t
docker run -d -...
> Top 2 nodes:
> [Node 838e3eb1-2149-468b-91c6-f8245f52d571] [Similarity score:             0.642404] Docker FAQ
Docker 加速相关配置

dockerfile 中常用加速

# 使用nvidia作为基础镜像
FROM nvidia/cuda:12.2.0-devel-ubuntu...
> [Node 116c7d5f-11f0-41e4-a6ae-1790250834ee] [Similarity score:             0.574852] CURRENT_UID=$(id -u):$

In [53]:
%%time

streaming_response = query_engine.query("docker怎么设置代理服务器")
streaming_response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ape:11434
Starting new HTTP connection (1): ape:11434
DEBUG:urllib3.connectionpool:http://ape:11434 "POST /api/embeddings HTTP/11" 200 None
http://ape:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 838e3eb1-2149-468b-91c6-f8245f52d571] [Similarity score:             0.682691] Docker FAQ
Docker 加速相关配置

dockerfile 中常用加速

# 使用nvidia作为基础镜像
FROM nvidia/cuda:12.2.0-devel-ubuntu...
> [Node e8d6f116-79df-48bc-b710-cb9180e56c6a] [Similarity score:             0.621747] {
  "repositories": [
    "app-server",
    "curl" # <--有了
  ]
}

docker 下 nginx 配置简单的静态网站

创建目录和...
> Top 2 nodes:
> [Node 838e3eb1-2149-468b-91c6-f8245f52d571] [Similarity score:             0.682691] Docker FAQ
Docker 加速相关配置

dockerfile 中常用加速

# 使用nvidia作为基础镜像
FROM nvidia/cuda:12.2.0-devel-ubuntu...
> [Node e8d6f116-79df-48bc-b710-cb9180e56c6a] [Similarity score:             0.621747] {
  "repositories": [


In [54]:
streaming_response.source_nodes[0]

NodeWithScore(node=TextNode(id_='838e3eb1-2149-468b-91c6-f8245f52d571', embedding=None, metadata={'document_id': 'FJ1ndY2MsoTWTPxWWaTcXBZonJl'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9849e2bb-5e16-4e80-880c-cebfa88a6e14', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'document_id': 'FJ1ndY2MsoTWTPxWWaTcXBZonJl'}, hash='97f16a5fa6648c0b10d7558a113b42637f8ab638e35d220000b1edf0f6642d56'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='116c7d5f-11f0-41e4-a6ae-1790250834ee', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='38ea7644d0fa218279b599e540d68d47665eb1a5499bf5af194218a923b6b0cd')}, text='Docker FAQ\nDocker 加速相关配置\n\ndockerfile 中常用加速\n\n# 使用nvidia作为基础镜像\nFROM nvidia/cuda:12.2.0-devel-ubuntu22.04\n\n# 设置时区环境变量\nENV TZ=Asia/Shanghai\n\n# 设置时区\n# 安装 python3/pip/jupyter相关\nRUN ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && echo $TZ > /etc/timezone \\\n    && sed -i s@/archi